## Scrape of a Wordpress site

In this notebook, I attempt to scrape a wordpress site using BeautifulSoup4. The objective is to aid the production team by providing an XML or JSON file that they can in turn import into a new Wordpress site. The result will save human hours by providing a simple summary of the contents of a page.

In [1]:
# First import the tools

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
# Read the page into python
r = requests.get('http://www.heckler.com/')
# print (r.text[0:1000]) # Verify that there is connection allowed

In [3]:
# Parse into Beautiful Soup
soup = bs(r.text,'html.parser')

In [4]:
# Find all elements of the articles

main_container = soup.find('div', attrs={'class':'cb-main'})
results = main_container.find_all('article')
first_result = results[0]
title = first_result.find('h2').text
author = ((first_result.find('div', attrs={'class':'cb-author'})).text)[1:]
date_updated = first_result.find('time')['datetime']
categories = (first_result.find_all('div', attrs={'class':'cb-category'}))[0].text
url = first_result.find('a')['href']
excerpt = (first_result.find('div', attrs={'class':'cb-excerpt'}).text)



In [5]:
# Going through all articles in the loaded page

records = []
for result in results:
    
    title = result.find('h2').text
    author = ((result.find('div', attrs={'class':'cb-author'})).text)[1:]
    date_updated = result.find('time')['datetime']
    categories = (result.find_all('div', attrs={'class':'cb-category'}))[0].text
    excerpt = result.find('div', attrs={'class':'cb-excerpt'}).text
    url = result.find('a')['href']
    records.append((title, author, date_updated, categories, excerpt, url))


In [6]:
# Loading records into a Pandas Dataframe

df = pd.DataFrame(records, columns=['title', 'author', 'date_updated', 'categories', 'excerpt', 'url'])
df['date_updated'] = pd.to_datetime(df['date_updated'])
df.to_csv('export.csv', index=False, encoding='utf-8')
